In [106]:
import datetime as dt
import utils as ut
import pandas as pd
import icharts as ic
import holoviews as hv
import base as b
import hvplot.pandas  # noqa
from functools import cache
from logger_settings import logger
from constants import *
from bokeh.plotting import figure, show, output_notebook
output_notebook()


test_date = dt.datetime.strptime("2024-01-01", "%Y-%m-%d")
SYMBOL = "NIFTY 50"
IC_SYMBOL = "NIFTY"
INTERVAL = ut.INTERVAL_MIN1
EXCHANGE = ut.EXCHANGE_NSE


TEST_START = dt.datetime.strptime("2024-01-02", "%Y-%m-%d")
TEST_END = dt.datetime.strptime("2024-01-10", "%Y-%m-%d")

train_dates, test_dates = ut.get_date_range(start_date=TEST_START, end_date=TEST_END, symbol=SYMBOL, interval=INTERVAL, exchange=EXCHANGE)
train_dates["expiry"] = pd.NA
train_dates["expiry"] = train_dates.apply(lambda row: ut.find_nclosest_expiry(SYMBOL, row.name, 1), axis=1)

train_dates['call_strike'] = train_dates.apply(lambda r: ut.get_strike_price_by_price(symbol=SYMBOL, expiry=r.expiry, timestamp=r.name.replace(hour=9, minute=15), option_type=OPTION_TYPE_CALL, price=30, exchange=EXCHANGE), axis=1)
train_dates['put_strike'] = train_dates.apply(lambda r: ut.get_strike_price_by_price(symbol=SYMBOL, expiry=r.expiry, timestamp=r.name.replace(hour=9, minute=15), option_type=OPTION_TYPE_PUT, price=30, exchange=EXCHANGE), axis=1)

# Get nifty candles for minute
# For first minute, get premium strike price for call and puts which is price at ~30 
# Draw them here 
# Try to match the pattern 

Loading BokehJS ...

In [102]:
settings = {
    "VOLUME_QUANTILE_THRESHOLD": 10 / 100, # Choose least this % volume from the given volumes
    "CANDLE_LENTH": 20,
    "MIN_GREEN_CANDLE_LENGTH": 3, # Minimum candles to be considered as pattern
    "RED_CANDLE_RATIO": 0.65,
    "GREEN_THRESHOLD": 0.05, # For candle to be considered green, percentage of average
    "RED_CANDLE_DIFF": 0.28, # Red candle size compared to green up move
    "MIN_QV_THRESHOLD": 3.8, # Minimum volume ratio to qualify for green candle
    "MAX_QV_THRESHOLD": 50, # Minimum volume ratio to qualify for green candle
    # Order Settings
    "quantity": 25,
    "TARGET_PC_MIN": 0.05,
    "TARGET_PC_MAX": .7,
    "STOPLOSS_PC": 0.03,
    "TARGET_AS_DIFF_PC": 0.5,
    "STOPLOSS_AS_DIFF_PC": 0.5,
    "STOPLOSS_MIN_CAP_PC": 0.10, # Stoploss PC w.r.t. buying price
}

from typing import Dict


class PeakStrategy(b.Strategy):
    def __init__(self, instrument: "Instrument", settings: Dict):
        super().__init__(instrument, settings)
        self.call_ticks: pd.DataFrame | None = None
        self.put_ticks: pd.DataFrame | None = None
        self.om = b.OrderManager()

    def calculate_data(self):
        self.put_ticks['qv'] = self.put_ticks['volume'].rolling(window=self.CANDLE_LENTH).quantile(self.VOLUME_QUANTILE_THRESHOLD)
        self.call_ticks['qv'] = self.call_ticks['volume'].rolling(window=int(self.CANDLE_LENTH / 1.5)).quantile(self.VOLUME_QUANTILE_THRESHOLD)
        self.put_ticks['qv_ratio'] = self.put_ticks.volume / self.put_ticks.qv
        self.call_ticks['qv_ratio'] = self.call_ticks.volume / self.call_ticks.qv
        self.put_ticks['qv_qualify'] = self.put_ticks.qv_ratio > self.MIN_QV_THRESHOLD
        self.put_ticks['prev_ratio'] = self.put_ticks.volume / self.put_ticks.volume.shift(1)
        self.put_ticks['prev1_ratio'] = self.put_ticks.volume / self.put_ticks.volume.shift(2)
        self.put_ticks['prev_qv_qualify'] = self.put_ticks.qv_qualify.shift(1)
        self.put_ticks['prev1_qv_qualify'] = self.put_ticks.qv_qualify.shift(2)
        self.put_ticks['red_qualify'] = (self.put_ticks.prev_qv_qualify | self.put_ticks.prev1_qv_qualify) & ((self.put_ticks.prev_ratio < self.RED_CANDLE_RATIO) | (self.put_ticks.prev1_ratio < self.RED_CANDLE_RATIO))
        self.put_ticks['wprice'] = (self.put_ticks.high + self.put_ticks.low + 2 * self.put_ticks.close) / 4
        self.call_ticks['wprice'] = (self.call_ticks.high + self.call_ticks.low + 2 * self.call_ticks.close) / 4
        self.put_ticks['cdiff'] = self.put_ticks.wprice.diff()
        self.put_ticks['is_green'] = self.put_ticks.cdiff > 0
        self.put_ticks['is_small'] = self.put_ticks.cdiff < self.put_ticks.cdiff.rolling(window=self.CANDLE_LENTH).mean() * self.GREEN_THRESHOLD

    def is_volume_match(self) -> bool:
        '''
        1. Must have high volume on the last green candle
        2. Must have low volume on the last red candle
        '''
        return self.put_ticks.iloc[-1].red_qualify

    def is_price_match(self):
        '''
        1. Price must be increasing until the last candle
        2. There must not be big gaps in the candles, that could mean that it's going to go further up
        '''
        last_tick = self.put_ticks.iloc[-1]
        last_n_ticks = self.put_ticks[-self.MIN_GREEN_CANDLE_LENGTH-1:-1]
        green_length = last_n_ticks.loc[last_n_ticks.is_green].shape[0]
        if green_length < self.MIN_GREEN_CANDLE_LENGTH:
            last_n_ticks = self.put_ticks[-self.MIN_GREEN_CANDLE_LENGTH-2:-1]
        if last_tick.is_green:
            return False
        if green_length < self.MIN_GREEN_CANDLE_LENGTH - 1:
            return False
        # elif green_length == self.MIN_GREEN_CANDLE_LENGTH:
        #     return True
        price_min, price_max = last_n_ticks.open.min(), last_n_ticks.close.max()
        red_ratio = (price_max - last_tick.close) / (price_max - price_min)
        if red_ratio > self.RED_CANDLE_DIFF:
            return False
        return True

    def entry_conditions(self) -> bool:
        last_tick = self.put_ticks.iloc[-1]
        last_n_ticks = self.put_ticks[-self.MIN_GREEN_CANDLE_LENGTH-1:-1]
        is_price_match = self.is_price_match()
        is_volume_match = self.is_volume_match()
        if not is_volume_match:
            return False
        if not is_price_match:
            return False
        print(f"matched at: {self.put_ticks.iloc[-1].timestamp}")
        return True

    def exit_conditions(self) -> bool:
        last_tick = self.call_ticks.iloc[-1]
        closep = last_tick.close
        if last_tick.timestamp.hour == 15 and last_tick.timestamp.minute == 28:
            return True, closep
        wprice = last_tick.wprice
        openp = last_tick.open
        for order in self.om.orders:
            if order.created_at == last_tick.timestamp:
                continue
            if (order.limit_price - openp) >= order.meta['sl']:
                return True, order.limit_price - order.meta['sl']
            if (wprice - order.limit_price) >= order.meta['tp']:
                return True, order.limit_price + order.meta['tp']
            if (openp - order.limit_price) >= order.meta['tp']:
                return True, order.limit_price + order.meta['tp']
            if (order.limit_price - closep) >= order.meta['sl']:
                return True, order.limit_price - order.meta['sl']
        return False, None

    def _process_tick(self, call_tick: Dict, put_tick: Dict) -> None:
        call_tick_df = pd.DataFrame(
            call_tick, index=[len(self.call_ticks) if self.call_ticks is not None else 0]
        )
        self.call_ticks = pd.concat([self.call_ticks, call_tick_df], ignore_index=True)

        put_tick_df = pd.DataFrame(
            put_tick, index=[len(self.put_ticks) if self.put_ticks is not None else 0]
        )
        self.put_ticks = pd.concat([self.put_ticks, put_tick_df], ignore_index=True)

    def next(self, call_tick: Dict, put_tick: Dict):
        self._process_tick(call_tick, put_tick)
        self.calculate_data()
        if not self.om.has_intrade_orders() and self.entry_conditions():
            price_diff = self.call_ticks.iloc[-5:].open.max() - self.call_ticks.iloc[-5:].close.min()
            qv_max = self.call_ticks.iloc[-5:].qv_ratio.max()
            target_pc = qv_max * (self.TARGET_PC_MAX - self.TARGET_PC_MIN ) / (self.MAX_QV_THRESHOLD - self.MIN_QV_THRESHOLD)
            target = self.call_ticks.iloc[-1].close * target_pc
            print(f"at: {self.call_ticks.iloc[-1].timestamp}, qm: {qv_max}, tar: {target}, tpc: {target_pc}")
            stoploss = self.call_ticks.iloc[-1].close - self.call_ticks.iloc[-1].low + self.call_ticks.iloc[-5:].low.std()
            stoploss = min(stoploss, target)
            order = b.Order(type=b.Order.TYPE_BUY, limit_price=self.call_ticks.iloc[-1].close, created_at=self.call_ticks.iloc[-1].timestamp, quantity=self.quantity, exchange_order_id=None, meta={'tp': target, 'sl': stoploss,})
            self.om.place_order(order)
        is_exit, eprice = self.exit_conditions()
        if self.om.has_intrade_orders() and is_exit:
            self.om.square_off_all_orders(index=self.call_ticks.iloc[-1].timestamp, last_price=eprice)


instrument = b.Instrument(name="NIFTY 22650 CALL 7 Mar 2024")

ps = PeakStrategy(instrument=instrument, settings=settings)

for index, row in train_dates.iterrows():
    call_df = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=row.expiry, cur_dt=index.date(), strike_price=row.call_strike, option_type=OPTION_TYPE_CALL)
    put_df = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=row.expiry, cur_dt=index.date(), strike_price=row.put_strike, option_type=OPTION_TYPE_PUT)
    for i in range(put_df.shape[0]):
        cdict = call_df.iloc[i].to_dict()
        cdict['timestamp'] = call_df.iloc[i].name
        pdict = put_df.iloc[i].to_dict()
        pdict['timestamp'] = put_df.iloc[i].name
        if not ps.om.has_intrade_orders() and (call_df.iloc[i].close < 18 or call_df.iloc[i].close > 40):
            old_price = call_df.iloc[i].close
            call_strike = ut.get_strike_price_by_price(symbol=SYMBOL, expiry=row.expiry, timestamp=call_df.iloc[i].name, option_type=OPTION_TYPE_CALL, price=30, exchange=EXCHANGE)
            if call_strike != row["call_strike"]:
                print(f"Changed call strike: {call_strike}, {row['call_strike']} old priced: {old_price} to new price: {call_df.iloc[i].close} at: {call_df.iloc[i].name}")
                row["call_strike"] = call_strike
                call_df = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=row.expiry, cur_dt=index.date(), strike_price=row.call_strike, option_type=OPTION_TYPE_CALL)
                # call_df = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=row.expiry, cur_dt=index.date(), strike_price=call_strike, option_type=OPTION_TYPE_CALL)
        if not ps.om.has_intrade_orders() and (put_df.iloc[i].close < 18 or put_df.iloc[i].close > 40):
            old_price = put_df.iloc[i].close
            put_strike = ut.get_strike_price_by_price(symbol=SYMBOL, expiry=row.expiry, timestamp=put_df.iloc[i].name, option_type=OPTION_TYPE_PUT, price=30, exchange=EXCHANGE)
            if put_strike != row["put_strike"]:
                print(f"Changed put strike: {put_strike}, {row['put_strike']} old priced: {old_price} to new price: {put_df.iloc[i].close} at: {put_df.iloc[i].name}")
                row["put_strike"] = put_strike
                put_df = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=row.expiry, cur_dt=index.date(), strike_price=row.put_strike, option_type=OPTION_TYPE_PUT)
                # put_df = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=row.expiry, cur_dt=index.date(), strike_price=put_strike, option_type=OPTION_TYPE_PUT)
        # cdict, pdict = pdict, cdict
        ps.next(cdict, pdict)
    break

Changed put strike: 21400, 21350 old priced: 17.8 to new price: 17.8 at: 2024-01-01 09:35:00
matched at: 2024-01-01 09:41:00
at: 2024-01-01 09:41:00, qm: 2.4906917499510093, tar: 0.6850750089286243, tpc: 0.03504219994519817
Changed put strike: 21450, 21400 old priced: 17.7 to new price: 17.7 at: 2024-01-01 10:41:00
Changed put strike: 21500, 21450 old priced: 17.9 to new price: 17.9 at: 2024-01-01 12:14:00
Changed put strike: 21550, 21500 old priced: 17.9 to new price: 17.9 at: 2024-01-01 14:02:00
Changed call strike: 22150, 22050 old priced: 42.5 to new price: 42.5 at: 2024-01-01 14:42:00
Changed put strike: 21600, 21550 old priced: 17.9 to new price: 17.9 at: 2024-01-01 14:44:00
matched at: 2024-01-01 14:48:00
at: 2024-01-01 14:48:00, qm: 8.713477851083884, tar: 2.8809171899416945, tpc: 0.12259222084858275
matched at: 2024-01-01 14:53:00
at: 2024-01-01 14:53:00, qm: 6.863885546196015, tar: 2.554271693787337, tpc: 0.09656981829063656
Changed call strike: 22100, 22150 old priced: 13.75

In [103]:
pcs = [order.pnl_pc for order in ps.om.closed_orders]
print(f"PnL Per Order: {sum(pcs) / len(pcs)}, success rate: {len([x for x in pcs if x > 0]) * 100 / len(pcs)}%")
pcs

PnL Per Order: 2.090075830488439, success rate: 66.66666666666667%


[3.5042199945198176, 12.259222084858274, -9.493214587912775]

In [105]:
print(ps.om.closed_orders[2].meta)
print(ps.om.closed_orders[2])

{'tp': 2.554271693787337, 'sl': 2.5109552585029276}
buy, at:2024-01-01 14:53:00, b:26.45, sqat:2024-01-01 15:01:00, s:23.93904474149707, pnl:-9.493214587912775


In [73]:
pd.set_option('display.max_rows', 200)
# ps.put_ticks[["timestamp", "qv", "qv_ratio", "qv_qualify", "prev_ratio", "prev1_ratio", "prev_qv_qualify", "prev1_qv_qualify", "red_qualify"]].head(200)
ps.call_ticks[["timestamp", "qv", "qv_ratio"]].head(200)

,timestamp,qv,qv_ratio
0,2024-01-01 09:15:00,NaN,NaN
1,2024-01-01 09:16:00,NaN,NaN
2,2024-01-01 09:17:00,NaN,NaN
3,2024-01-01 09:18:00,NaN,NaN
4,2024-01-01 09:19:00,NaN,NaN
5,2024-01-01 09:20:00,NaN,NaN
6,2024-01-01 09:21:00,NaN,NaN
7,2024-01-01 09:22:00,NaN,NaN
8,2024-01-01 09:23:00,NaN,NaN
9,2024-01-01 09:24:00,NaN,NaN


In [74]:
# call_df = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=train_dates.iloc[0].expiry, cur_dt=train_dates.iloc[0].name.date(), strike_price=train_dates.iloc[0].call_strike, option_type=OPTION_TYPE_CALL)
# put_df = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=train_dates.iloc[0].expiry, cur_dt=train_dates.iloc[0].name.date(), strike_price=train_dates.iloc[0].put_strike, option_type=OPTION_TYPE_PUT)
call_df = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=train_dates.iloc[0].expiry, cur_dt=train_dates.iloc[0].name.date(), strike_price=22150, option_type=OPTION_TYPE_CALL)
put_df = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=train_dates.iloc[0].expiry, cur_dt=train_dates.iloc[0].name.date(), strike_price=21500, option_type=OPTION_TYPE_PUT)

ut.create_candlestick_plot(call_df, title='Call')
ut.create_candlestick_plot(put_df, title='Put')

GridPlot(id='p5228', ...)

In [40]:
ps.put_ticks.hvplot(x='timestamp', y='cdiff')

:Curve   [timestamp]   (cdiff)

In [39]:
pd.set_option('display.max_rows', 200)
ps.put_ticks[["timestamp", "qv", "qv_ratio", "qv_qualify", "prev_ratio", "prev1_ratio", "prev_qv_qualify", "prev1_qv_qualify", "red_qualify"]].head(200)

,timestamp,qv,qv_ratio,qv_qualify,prev_ratio,prev1_ratio,prev_qv_qualify,prev1_qv_qualify,red_qualify
0,2024-01-02 09:15:00,NaN,NaN,False,NaN,NaN,NaN,NaN,False
1,2024-01-02 09:16:00,NaN,NaN,False,1.115373,NaN,False,NaN,False
2,2024-01-02 09:17:00,NaN,NaN,False,0.533067,0.594568,False,False,False
3,2024-01-02 09:18:00,NaN,NaN,False,1.039191,0.553958,False,False,False
4,2024-01-02 09:19:00,NaN,NaN,False,1.498347,1.557068,False,False,False
5,2024-01-02 09:20:00,NaN,NaN,False,1.076816,1.613444,False,False,False
6,2024-01-02 09:21:00,NaN,NaN,False,0.970175,1.044700,False,False,False
7,2024-01-02 09:22:00,NaN,NaN,False,0.793648,0.769978,False,False,False
8,2024-01-02 09:23:00,NaN,NaN,False,0.556968,0.442037,False,False,False
9,2024-01-02 09:24:00,NaN,NaN,False,0.891347,0.496452,False,False,False
